$\DeclareMathOperator{\trace}{tr}$

# Task 4 

# Libraries used

In [3]:
import numpy as np

## First moves

This task involves one of the most significant near term quantum algorithm: **Variational Quantum Eigensolver** (VQE). The issue at stake is to calculate the lowest eigenvalues of the following matrix:

$$ \begin{pmatrix}
0 & 0 & 0 & 0\\
0 & -1 & 1 & 0\\
0 & 1 & -1 & 0\\
0 & 0 & 0 & 0
\end{pmatrix}
$$

Let outline first the steps we will go through in order to solve this task.

1. Calculating the Hamiltonian decomposition
2. Proposing an Ansatz
3. Executing the circuit

We will now continue covering all the topics above. 



## Calculating the Hamiltonian decomposition

*This section is based entirely on the article published by Michael Goerz, check it [here](https://michaelgoerz.net/notes/decomposing-two-qubit-hamiltonians-into-pauli-matrices.html).*

We can decompose the matrix provided into a linear combination of the constituent parts of a basis of the 4x4 Hermitian matrices. The basis we shall consider is not other but the one generated by the Kronecker product of the Pauli Matrices and the Identity. We will now list all the 2x2 matrices involved which will generate our 4x4 Hermitian matrices basis:

$$
\sigma_{0} = Id = \begin{pmatrix}
1 & 0\\
0 & 1
\end{pmatrix} $$ 

$$
\sigma_{1} = \sigma_{X} = \begin{pmatrix}
0 & 1\\
1 & 0
\end{pmatrix} $$ 

$$\sigma_{2} = \sigma_{Y} = \begin{pmatrix}
0 & -i\\
i & 0
\end{pmatrix} $$ 

$$
\sigma_{3} = \sigma_{Z} = \begin{pmatrix}
1 & 0\\
0 & -1
\end{pmatrix}
$$

Then, the basis will just be:

$$  
\mathcal{B} = \{ \sigma_{i} \otimes \sigma_{j} : 0 \leq i,j \leq 3 \}
$$

A decomposition of an arbitrary 4x4 matrix $F$ in terms of elements of $\mathcal{B}$ could be easily calculated as the following expression<sup>1</sup> shows:
$$ 
F = \sum_{i,j=1,x,y,z} a_{i,j} \left( \sigma_i \otimes \sigma_j \right),
\quad
a_{i,j} = \frac{1}{4} \trace \left[\left( \sigma_i \otimes \sigma_j \right) H \right]
$$

In [12]:
def product_HilbertSchmidt(A, B):
    """Hilbert-Schmidt-Product of two matrices A and B"""
    return (np.dot(A.conjugate().transpose(), B)).trace()


def c2s(c):
    """Return a string representation of a complex number c"""
    if c == 0.0:
        return "0"
    if c.imag == 0:
        return "%g" % c.real
    elif c.real == 0:
        return "%gj" % c.imag
    else:
        return "%g+%gj" % (c.real, c.imag)

def decompose(H):
    """Decompose Hermitian 4x4 matrix H into Pauli matrices"""
    from numpy import kron
    sx = np.array([[0, 1],  [ 1, 0]], dtype=np.complex128)
    sy = np.array([[0, -1j],[1j, 0]], dtype=np.complex128)
    sz = np.array([[1, 0],  [0, -1]], dtype=np.complex128)
    id = np.array([[1, 0],  [ 0, 1]], dtype=np.complex128)
    S = [id, sx, sy, sz]
    labels = ['I', 'sigma_x', 'sigma_y', 'sigma_z']
    for i in range(4):
        for j in range(4):
            label = labels[i] + ' \otimes ' + labels[j]
            a_ij = 0.25 * product_HilbertSchmidt(kron(S[i], S[j]), H)
            if a_ij != 0.0:
                print("%s\t*\t( %s )" % (c2s(a_ij), label))

In [13]:
A = np.array([[0,0,0,0], [0,-1,1,0], [0,1,-1,0], [0,0,0,0]])
decompose(A)

-0.5	*	( I \otimes I )
0.5	*	( sigma_x \otimes sigma_x )
0.5	*	( sigma_y \otimes sigma_y )
0.5	*	( sigma_z \otimes sigma_z )


<sup>1</sup> The coefficient $1/4$ is a consequence of the fact that Pauli-matrices and the identity are not normalized as: $ \lVert\sigma_i\rVert = \sqrt{\trace\left[ \sigma_i^\dagger \sigma_i \right]} = \sqrt{2}$.